In [1]:
# Instalacion de las dependencis (developed from Google Colab)
! pip install -q scikit-learn matplotlib tensorflow numpy pandas

# Librerías a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split


In [2]:
# Los datos originales se obtienen de Google Drive. Se establece una conexión
drive.mount('/content/drive')

# Lectura del fichero
with open('/content/drive/My Drive/TFM/CHEMBL221_COX1Prostaglandin_Desc.csv', 'r') as f:
  original_data = pd.read_csv(f)

# Cierre de la conexión con Google Drive
drive.flush_and_unmount()

Mounted at /content/drive


In [16]:
# Preparación de los datos

# Eliminación de las 7 primeras columnas que no son de utilidad para nuestro propósito
work_data = original_data.drop(original_data.iloc[:, 0:7], axis = 1)

# Recogida de la columna Activity (el resultado a estudiar) y eliminación de la misma

activity = work_data['Activity'] 
work_data = work_data.drop('Activity', axis = 1) # Eliminación del resto de datos


In [17]:
# Eliminación de columnas con baja varianza (threshold = 1)

lvf = VarianceThreshold(threshold = 1) 
lvf.fit(work_data)

# Recogemos las columnas donde su varianza está por debajo del thresold establecido
lvc = [column for column in work_data.columns 
                    if column not in work_data.columns[lvf.get_support()]]

cr = [i.strip() for i in lvc] # Columnas a borrar
work_data = work_data.drop(cr, axis=1) # Eliminación de columnas con baja varianza (columnas constantes tienen varianza 0)

# Eliminación de columnas con alta correlacion (>= 0.8)
cm = work_data.corr().abs() # Matriz positiva de correlación

# Obtención de los elementos de la matriz por encima de la diagonal de la matriz de correlación
upper_triangle = cm.where(np.triu(np.ones(cm.shape), k=1).astype(bool))

# Selección de las columnas donde la correlación sea >=0.8
cr = [column for column in upper_triangle.columns if any(upper_triangle[column] >= 0.8)]
# Eliminación de las columnas con alta correlación
work_data = work_data.drop(cr, axis=1) 



In [18]:
# Obtain Training and Test sets (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(work_data, activity, test_size=0.20, random_state = 19680925, stratify=activity)

In [19]:
# Escritura de los ficheros obtenidos
# Serán almacenados en Google Drive
drive.mount('/content/drive', force_remount = True)

X_train.to_csv('/content/drive/My Drive/TFM/X_train.csv', index = False)
y_train.to_csv('/content/drive/My Drive/TFM/y_train.csv', index = False)
X_test.to_csv('/content/drive/My Drive/TFM/X_test.csv', index = False)
y_test.to_csv('/content/drive/My Drive/TFM/y_test.csv', index = False)


drive.flush_and_unmount()

Mounted at /content/drive
